In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
import warnings
import pickle
from sklearn.svm import SVR
import lightgbm as lgb
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor
from sklearn.externals import joblib
from sklearn.model_selection import RandomizedSearchCV
import time
import random 
import xgboost as xgb
warnings.filterwarnings('ignore')

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
!mkdir Case_study_models
!mkdir Case_study_submissions

In [ ]:
with open('/content/drive/MyDrive/Case_study mercedes_Manufacturing/data/df_train_top100.pkl','rb') as f:
  train = pickle.load(f)

with open('/content/drive/MyDrive/Case_study mercedes_Manufacturing/data/df_test_top100.pkl','rb') as k:
  test = pickle.load(k)  

with open('/content/drive/MyDrive/Case_study mercedes_Manufacturing/data/df_train_FE.pkl','rb') as f:
  train_FE = pickle.load(f)

with open('/content/drive/MyDrive/Case_study mercedes_Manufacturing/data/df_test_FE.pkl','rb') as k:
  test_FE = pickle.load(k) 

with open('/content/drive/MyDrive/Case_study mercedes_Manufacturing/data/df_train_250+labelenc.pkl','rb') as f:
  train_original = pickle.load(f)

with open('/content/drive/MyDrive/Case_study mercedes_Manufacturing/data/df_test_250+labelenc.pkl','rb') as k:
  test_original = pickle.load(k)  

In [ ]:
train_original.head(5)

,ID,X0,X1,X2,X3,X5,X6,X8,X10,X12,X13,X14,X16,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X30,X32,X34,X36,X37,X38,X40,X41,X43,X45,X46,X47,X49,X50,X51,X52,X55,...,X329,X331,X333,X334,X335,X336,X338,X340,X341,X342,X343,X344,X345,X346,X349,X350,X351,X353,X354,X355,X356,X357,X359,X360,X361,X365,X366,X369,X373,X374,X375,X376,X377,X379,X380,X382,X383,X384,X385,y
0,0,32,23,17,0,24,9,14,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,130.81
1,6,32,21,19,4,28,11,14,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,88.53
2,7,20,24,34,2,27,9,23,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,76.26
3,9,20,21,34,5,27,11,4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,80.62
4,13,20,23,34,5,12,3,13,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,78.02


In [ ]:
features=list(train_original.columns)
import pickle
with open('/content/features.pkl', 'wb') as f:
  pickle.dump(features, f)

In [ ]:
sample_submission=pd.read_csv("/content/drive/MyDrive/Case_study mercedes_Manufacturing/sample_submission.csv.zip")
df_test=pd.read_csv("/content/drive/MyDrive/Case_study mercedes_Manufacturing/test.csv")

In [ ]:
train_FE.shape , test_FE.shape

((4209, 278), (4209, 277))

In [ ]:
train_FE[: ,277]

array([130.81,  88.53,  76.26, ..., 109.22,  87.48, 110.85])

In [ ]:
X=train[: ,0:291]
Y=train[: ,291]
X_FE=train_FE[: ,0:277]
Y_FE=train_FE[: ,277]
X_original=train_original.drop("y" , axis=1)
Y_original=train_original["y"]

In [ ]:
def train_model(   model ,X , Y  , param , file_name  ,test=test , df_test=df_test):
  """
  in given function we are fitting train data on ML model 
  we are using Randomized Search CV with 5 fold 
  finally we get best model then we train model on X and Y   
  """
  start = time.time()
  param_list=list(param.keys())
  model_cv = RandomizedSearchCV(model, param_distributions=param)
  #best param from Randomized Search CV and training on X and Y
  best_model=model_cv.fit(X , Y)
  #R2 score on train data
  train_r2=r2_score(Y ,best_model.predict(X))

  line="*"*35
  print(line)
  #printing best param 
  for i in param_list:
    print(f"best_model param {i} :" ,best_model.best_estimator_.get_params()[i])
  print(line)

  print(f"train R2 score: {train_r2}")
  #predicting on test test
  y_pred_test=best_model.predict(test)
  
  #storing predicted Y and test ID in submission dataframe
  submission_df=pd.DataFrame({"ID" :df_test["ID"] ,"y":y_pred_test})
  elapsed = time.time() - start
  print(line)
  print(f"time required: {elapsed}")
  print(line)
  print("submission file saved succesfully!!!")
  submission_df.to_csv("/content/Case_study_submissions/submission_"+file_name ,index=False)
  print("best model saved succesfully!!!")
  #dumping model to pickle file
  joblib_file = "/content/Case_study_models/best_model_"+file_name 
  joblib.dump(best_model, joblib_file)
  print(line)
  return best_model 

Lasso Regression

In [ ]:
reg = linear_model.Lasso()
param={"max_iter" : [1000 , 1500 ,2000]}
best_model=train_model(reg , X , Y , param ,"reg_100" )

***********************************
best_model param max_iter : 1000
***********************************
train R2 score: 0.3681997917390706
***********************************
time required: 0.8714864253997803
***********************************
submission file saved succesfully!!!
best model saved succesfully!!!
***********************************


In [ ]:
reg = linear_model.Lasso()
param={"max_iter" : [1000 , 1500 ,2000]}
best_model=train_model(reg , X_FE , Y_FE , param ,"reg_FE" , test_FE)

***********************************
best_model param max_iter : 1000
***********************************
train R2 score: 0.4383464311567631
***********************************
time required: 0.7414767742156982
***********************************
submission file saved succesfully!!!
best model saved succesfully!!!
***********************************


In [ ]:
reg = linear_model.Lasso()
param={"max_iter" : [1000 , 1500 ,2000]}
best_model=train_model(reg , X_original , Y_original , param ,"reg_original" , test_original)

***********************************
best_model param max_iter : 1000
***********************************
train R2 score: 0.37284797026249794
***********************************
time required: 0.5467307567596436
***********************************
submission file saved succesfully!!!
best model saved succesfully!!!
***********************************


DT

In [ ]:
DecisionTreeRegressor(criterion = "mse")

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [ ]:
DT=  DecisionTreeRegressor(criterion = "mse")
param={"max_depth":[5 , 10 , 15 , 20]  ,
       "min_samples_split":[2 , 4 , 8 , 10] ,
       "min_samples_leaf":[1 , 5 , 10 , 15] }
best_model_DT =train_model(DT , X , Y ,  param , "DT_100")

***********************************
best_model param max_depth : 5
best_model param min_samples_split : 2
best_model param min_samples_leaf : 10
***********************************
train R2 score: 0.5806998541203842
***********************************
time required: 2.43764066696167
***********************************
submission file saved succesfully!!!
best model saved succesfully!!!
***********************************


In [ ]:
DT=  DecisionTreeRegressor(criterion = "mse")
param={"max_depth":[5 , 10 , 15 , 20]  ,
       "min_samples_split":[2 , 4 , 8 , 10] , 
       "min_samples_leaf":[1 , 5 , 10 , 15] }
best_model =train_model(DT , X_FE , Y_FE ,  param , "DT_FE" , test_FE)

***********************************
best_model param max_depth : 5
best_model param min_samples_split : 2
best_model param min_samples_leaf : 10
***********************************
train R2 score: 0.5860591902804202
***********************************
time required: 5.961425065994263
***********************************
submission file saved succesfully!!!
best model saved succesfully!!!
***********************************


In [ ]:
DT=  DecisionTreeRegressor(criterion = "mse")
param={"max_depth":[5 , 10 , 15 , 20]  ,
       "min_samples_split":[2 , 4 , 8 , 10] ,
       "min_samples_leaf":[1 , 5 , 10 , 15] }
best_model=train_model(DT , X_original , Y_original , param ,"DT_original" , test_original)

***********************************
best_model param max_depth : 10
best_model param min_samples_split : 4
best_model param min_samples_leaf : 10
***********************************
train R2 score: 0.6172990666572
***********************************
time required: 3.419506788253784
***********************************
submission file saved succesfully!!!
best model saved succesfully!!!
***********************************


RF

In [ ]:
RandomForestRegressor()

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
RF=  RandomForestRegressor(criterion = "mse")
param={"max_depth":[5 , 10 , 15 , 20]  ,
       "min_samples_split":[2 , 4 , 8 , 10] , 
       "min_samples_leaf":[1 , 5 , 10 , 15] }
best_model=train_model(RF , X , Y ,  param ,"RF_100")

***********************************
best_model param max_depth : 5
best_model param min_samples_split : 2
best_model param min_samples_leaf : 5
***********************************
train R2 score: 0.5870579979911417
***********************************
time required: 143.54842281341553
***********************************
submission file saved succesfully!!!
best model saved succesfully!!!
***********************************


In [ ]:
RF=  RandomForestRegressor(criterion = "mse")
param={"max_depth":[5 , 10 , 15 , 20]  ,
       "min_samples_split":[2 , 4 , 8 , 10] , 
       "min_samples_leaf":[1 , 5 , 10 , 15] }
best_model=train_model(RF , X_FE , Y_FE ,  param ,"RF_FE" , test_FE)

***********************************
best_model param max_depth : 5
best_model param min_samples_split : 8
best_model param min_samples_leaf : 15
***********************************
train R2 score: 0.5898502264431038
***********************************
time required: 318.28091192245483
***********************************
submission file saved succesfully!!!
best model saved succesfully!!!
***********************************


In [ ]:
RF=  RandomForestRegressor(criterion = "mse")
param={"max_depth":[5 , 10 , 15 , 20]  ,
       "min_samples_split":[2 , 4 , 8 , 10] ,
       "min_samples_leaf":[1 , 5 , 10 , 15] }
best_model=train_model(RF , X_original , Y_original , param ,"RF_original" , test_original)

***********************************
best_model param max_depth : 5
best_model param min_samples_split : 4
best_model param min_samples_leaf : 15
***********************************
train R2 score: 0.5838728495163685
***********************************
time required: 169.87348866462708
***********************************
submission file saved succesfully!!!
best model saved succesfully!!!
***********************************


XGB

In [ ]:
XGBRegressor()

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
XGB= XGBRegressor(objective ='reg:linear', verbosity = 0, random_state=42)
param={"max_depth":[5 , 10 , 15 , 20],
       "min_samples_split":[2 , 4 , 8 , 10],
       "min_samples_leaf":[1 , 5 , 10 , 15],
       "learning_rate":[0.1 , 0.01 , 0.001 ],
       "n_estimators":[100 , 200 , 300]}
best_model =train_model(XGB , X , Y ,  param , "XGB_100")

***********************************
best_model param max_depth : 5
best_model param min_samples_split : 4
best_model param min_samples_leaf : 1
best_model param learning_rate : 0.1
best_model param n_estimators : 100
***********************************
train R2 score: 0.6881652976964507
***********************************
time required: 410.6151866912842
***********************************
submission file saved succesfully!!!
best model saved succesfully!!!
***********************************


In [ ]:
XGB= XGBRegressor(objective ='reg:linear', verbosity = 0, random_state=42)
param={"max_depth":[5 , 10 , 15 , 20],
       "min_samples_split":[2 , 4 , 8 , 10],
       "min_samples_leaf":[1 , 5 , 10 , 15],
       "learning_rate":[0.1 , 0.01 , 0.001 ],
       "n_estimators":[100 , 200 , 300]}
best_model =train_model(XGB , X_FE , Y_FE ,  param , "XGB_FE" , test_FE)

***********************************
best_model param max_depth : 20
best_model param min_samples_split : 4
best_model param min_samples_leaf : 1
best_model param learning_rate : 0.01
best_model param n_estimators : 300
***********************************
train R2 score: 0.6492724296404032
***********************************
time required: 710.7077991962433
***********************************
submission file saved succesfully!!!
best model saved succesfully!!!
***********************************


In [ ]:
XGB= XGBRegressor(objective ='reg:linear', verbosity = 0, random_state=42)
param={"max_depth":[5 , 10 , 15 , 20],
       "min_samples_split":[2 , 4 , 8 , 10],
       "min_samples_leaf":[1 , 5 , 10 , 15],
       "learning_rate":[0.1 , 0.01 , 0.001 ],
       "n_estimators":[100 , 200 , 300]}
best_model=train_model(XGB , X_original , Y_original , param ,"XGB_original" , test_original)

***********************************
best_model param max_depth : 20
best_model param min_samples_split : 8
best_model param min_samples_leaf : 15
best_model param learning_rate : 0.1
best_model param n_estimators : 100
***********************************
train R2 score: 0.998286012791315
***********************************
time required: 390.90516567230225
***********************************
submission file saved succesfully!!!
best model saved succesfully!!!
***********************************


# Custom Ensemble with different models





In [ ]:
with open('/content/drive/MyDrive/Case_study mercedes_Manufacturing/data/df_train_top100.pkl','rb') as f:
  train = pickle.load(f)

with open('/content/drive/MyDrive/Case_study mercedes_Manufacturing/data/df_test_top100.pkl','rb') as k:
  test = pickle.load(k)  

In [ ]:
def custom_ensemble(train ,test ,K ,data_point, dim  ,model , meta_model  ,df_test=df_test):
  start = time.time()
  test_pred_list=[] ;test_data_pred_list=[];test_final_pred_list=[]
  line="*"*40
  train_data ,test_data=train_test_split(train , test_size=0.20 , random_state=45)   # splitting train data into train_X & test_X  with test_size=0.20
  D1 , D2 =train_test_split(train_data , test_size=0.50 , random_state=45)           #splitting train_X  into D1 & D2 with 50% split

  lasso_reg= joblib.load("/content/drive/MyDrive/Case_study mercedes_Manufacturing/models/best_model_reg_"+data)   #loading best models
  DT=joblib.load("/content/drive/MyDrive/Case_study mercedes_Manufacturing/models/best_model_DT_"+data)
  RF=joblib.load("/content/drive/MyDrive/Case_study mercedes_Manufacturing/models/best_model_RF_"+data)
  XGB=joblib.load("/content/drive/MyDrive/Case_study mercedes_Manufacturing/models/best_model_XGB_"+data)
  models_list= [lasso_reg ,DT , RF , XGB] ; k_datasets=[] ;models= [lasso_reg ,DT , RF , XGB] 
  for i in range(0 , K):                                                             # we are making K datasets from D1
    k_datasets.append(D1[np.random.choice(D1.shape[0],data_point, replace=False)])

  k_models=list(random.choices(models_list, k=K))                                    # k no. of best  models 
  test_D2=D2[: ,0:dim];test_Y_D2=D2[: ,dim] ;X_test=test_data[: ,0:dim];Y_test=test_data[: ,dim] ; X_final=test[: ,0:dim]

  for i  ,j in tqdm(enumerate(k_datasets)):                                          # training each model with each  k datasets
    data=k_datasets[i]
    X=data[: ,0:dim] ; Y=data[: ,dim]
    model=k_models[i].fit(X , Y)
    test_pred=model.predict(test_D2) ;test_pred_list.append(test_pred) ;test_data_pred=model.predict(X_test) ;test_data_pred_list.append(test_data_pred) 
    test_final_pred=model.predict(X_final) ; test_final_pred_list.append(test_final_pred)

  test_pred_stack_D2=np.vstack((test_pred_list)).T ;test_data_stack=np.vstack((test_data_pred_list)).T # stacking K predicted target variable Y
  test_final_stack=np.vstack((test_final_pred_list)).T
  model_meta=meta_model.fit(test_pred_stack_D2 ,test_Y_D2)                                              #training meta model with stacked target variable 
  print()
  print(line)
  print(f"train R2 score is {r2_score(test_Y_D2 ,model_meta.predict(test_pred_stack_D2))}")              # R2 score on test data
  print(line)
  print(f"test data R2 score is {r2_score(Y_test ,model_meta.predict(test_data_stack))}")
  print(line)
  print(f"time required: {time.time() - start}")
  print(line)
  y_pred_test=model_meta.predict(test_final_stack)                                                      # predicting target variable on meta data model
  submission_df=pd.DataFrame({"ID" :df_test["ID"] ,"y":y_pred_test})
  return submission_df

Lasso K=10 , data_point=1000 , data=top100

In [ ]:
model=linear_model.Lasso()
submission_CE_100=custom_ensemble(train ,test ,
                                  K=10 , 
                                  data_point=1000,
                                  dim=291 ,
                                  model=model1,meta_model=model)

10it [09:02, 54.27s/it]


****************************************
train R2 score is 0.5785281666681744
****************************************
test data R2 score is 0.588579736704218
****************************************
time required: 542.8286216259003
****************************************


In [ ]:
submission_CE_100.to_csv("/content/Case_study_submissions/submission_"+"CE_100",index=False)

meta_model=Lasso , k=15 , datapoint=1000 , data=train_FE

In [ ]:
model=linear_model.Lasso()
submission_CE_FE=custom_ensemble(train_FE ,test_FE , K=15 , data_point=1000, dim=277 ,data="FE",meta_model=model)

15it [14:27, 57.84s/it]


****************************************
train R2 score is 0.5862371325028449
****************************************
test data R2 score is 0.5649546175941406
****************************************
time required: 867.7395405769348
****************************************


In [ ]:
submission_CE_FE.to_csv("/content/Case_study_submissions/submission_"+"CE_FE",index=False)

In [ ]:
train_original_np=train_original.to_numpy(dtype ='float32')
test_original_np=test_original.to_numpy(dtype ='float32')

In [ ]:
model=linear_model.Lasso()
submission_CE_original=custom_ensemble(train_original_np ,test_original_np , K=15 , data_point=1000, dim=257 ,data="original",meta_model=model)

15it [10:43, 42.88s/it]


****************************************
train R2 score is 0.5860088689369656
****************************************
test data R2 score is 0.5904494966306111
****************************************
time required: 643.3167154788971
****************************************


In [ ]:
submission_CE_original.to_csv("/content/Case_study_submissions/submission_"+"CE_original",index=False)

# Custom Ensemble with Same models

In [ ]:
def custom_ensemble(train ,test ,K ,data_point, dim  , model ,meta_model  ,df_test=df_test):
  start = time.time()
  test_pred_list=[] ;test_data_pred_list=[];test_final_pred_list=[] ;k_datasets=[]
  line="*"*40
  train_data ,test_data=train_test_split(train , test_size=0.20 , random_state=45)     # splitting train data into train_X & test_X  with test_size=0.20
  D1 , D2 =train_test_split(train_data , test_size=0.50 , random_state=45)             # training meta model with stacked target variable 
  for i in range(0 , K):
    k_datasets.append(D1[np.random.choice(D1.shape[0],data_point, replace=False)])
  test_D2=D2[: ,0:dim];test_Y_D2=D2[: ,dim] ;X_test=test_data[: ,0:dim];Y_test=test_data[: ,dim] ;X_final=test[: ,0:dim]

  for i  ,j in tqdm(enumerate(k_datasets)):                                             # training each model with each  k datasets
    data=k_datasets[i]
    X=data[: ,0:dim] ; Y=data[: ,dim]
    model1=model.fit(X , Y)

    test_pred=model1.predict(test_D2)
    test_pred_list.append(test_pred)

    test_data_pred=model1.predict(X_test)
    test_data_pred_list.append(test_data_pred)

    test_final_pred=model1.predict(X_final)
    test_final_pred_list.append(test_final_pred)

  test_pred_stack_D2=np.vstack((test_pred_list)).T ; test_data_stack=np.vstack((test_data_pred_list)).T  #stacking predicted values from K datasets
  test_final_stack=np.vstack((test_final_pred_list)).T

  model_meta=meta_model.fit(test_pred_stack_D2 ,test_Y_D2)                              # training meta model on predicted values
  print()
  print(line)
  print(f"train R2 score is {r2_score(test_Y_D2 ,model_meta.predict(test_pred_stack_D2))}")   # R2 score on test data
  print(line)
  print(f"test data R2 score is {r2_score(Y_test ,model_meta.predict(test_data_stack))}")
  print(line)
  print(f"time required: {time.time() - start}")
  print(line)
  y_pred_test=model_meta.predict(test_final_stack)                                      # predicting target variable on meta data model
  submission_df=pd.DataFrame({"ID" :df_test["ID"] ,"y":y_pred_test})
  return submission_df

In [ ]:
model=linear_model.Lasso()
data="FE"
regressor = DecisionTreeRegressor()
DT=joblib.load("/content/drive/MyDrive/Case_study mercedes_Manufacturing/models/best_model_DT_"+data)

submission_df=custom_ensemble(train_FE ,test_FE ,
                              K=25 , 
                              data_point=1500,
                              dim=277 ,
                              model=DT ,
                              meta_model=model)

25it [00:48,  1.93s/it]


****************************************
train R2 score is 0.5837857901023645
****************************************
test data R2 score is 0.5685574822080066
****************************************
time required: 48.28375959396362
****************************************


0.53628
0.53834

In [ ]:
submission_df.to_csv("/content/Case_study_submissions/submission_"+"CEs_FE",index=False)

In [ ]:
model=linear_model.Lasso()
data="original"
regressor = DecisionTreeRegressor()
RF=joblib.load("/content/drive/MyDrive/Case_study mercedes_Manufacturing/models/best_model_RF_"+data)

submission_CE_FE=custom_ensemble(train_original_np ,test_original_np,
                                 K=25 , 
                                 data_point=1500,
                                 dim=257 ,
                                 model=DT ,
                                 meta_model=model)

25it [00:24,  1.00it/s]


****************************************
train R2 score is 0.5854309442183603
****************************************
test data R2 score is 0.584196368699575
****************************************
time required: 24.982601165771484
****************************************


In [ ]:
submission_df.to_csv("/content/Case_study_submissions/submission_"+"CEs_original",index=False)

# Avg modeling

In [ ]:
def avg_mode(train ,test ,dim  ,model_1):
  start = time.time()
  num_boost_rounds = 1250 
  X=train[: ,0:dim]
  Y=train[: ,dim]
  y_mean = np.mean(Y)
  model_1.fit(X, Y) #train model one
  joblib_file = "/content/"+"avg_model_XGB"
  joblib.dump(model_1, joblib_file)
  y_tr_pred1 = model_1.predict(X)
  p1 = model_1.predict(test)       #predict on model one
  dtrain = xgb.DMatrix(X , Y)
  dtest = xgb.DMatrix(test)      
  xgb_params = {'eta': 0.0045,'max_depth': 4,'subsample': 0.93, 'eval_metric': 'rmse',
                    'base_score': y_mean, 'colsample_bytree': 0.7, 'seed': 2}
  model_2 = xgb.train(xgb_params, dtrain, num_boost_round=num_boost_rounds)
  joblib_file = "/content/"+"avg_model_2"
  joblib.dump(model_2, joblib_file)
  y_tr_pred2 = model_2.predict(dtrain)
  p2 = model_2.predict(dtest)       #predict on model two
  avg_pred = (y_tr_pred1+y_tr_pred2)/2
  print("*" *40)
  print(r2_score(Y, avg_pred))
  print("*" *40)
  final_pred = (p1+p2)/2
  submission_df=pd.DataFrame({"ID" :df_test["ID"] ,"y":final_pred})
  print("*" *40)
  print(f"time required: {time.time() - start}")
  print("*" *40)

  return submission_df

In [ ]:
XGB= joblib.load("/content/drive/MyDrive/Case_study mercedes_Manufacturing/models/best_model_XGB_"+"original")

[15:57:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


best_model param max_depth : 20

best_model param min_samples_split : 8

best_model param min_samples_leaf : 15

best_model param learning_rate : 0.1

best_model param n_estimators : 100

In [ ]:
train_original_np=train_original.to_numpy(dtype ='float32')
test_original_np=test_original.to_numpy(dtype ='float32')

In [ ]:
# XGB=XGBRegressor(max_depth = 20 ,min_samples_split = 8 , min_samples_leaf = 15 , learning_rate = 0.1 ,n_estimators =100)
# joblib_file = "/content/drive/MyDrive/Case_study mercedes_Manufacturing/models_"+"avg_model_XGB"
# joblib.dump(XGB, joblib_file)

['/content/drive/MyDrive/Case_study mercedes_Manufacturing/models_avg_model_XGB']

In [ ]:
submission_df.to_csv("/content/Case_study_submissions/submission_"+"AM_original",index=False)

In [ ]:
model_1 = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=3, gpu_id=0,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=2,
             min_child_weight=1,
             n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=42,
             reg_alpha=10.0, reg_lambda=1, scale_pos_weight=1, subsample=0.93,
             tree_method='exact', validate_parameters=1)

submission_df=avg_mode(train_original_np ,test_original_np ,257  ,model_1 ) 

[15:57:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
****************************************
0.6135039252519263
****************************************
****************************************
time required: 23.007107734680176
****************************************


In [ ]:
model_1 = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=3, gpu_id=0,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=2,
             min_child_weight=1,
             n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=42,
             reg_alpha=10.0, reg_lambda=1, scale_pos_weight=1, subsample=0.93,
             tree_method='exact', validate_parameters=1)

submission_df=avg_mode(train_FE ,test_FE ,277  ,model_1 ) 

[16:00:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
****************************************
0.6376203874798019
****************************************
****************************************
time required: 27.55612325668335
****************************************


In [ ]:
submission_df.to_csv("/content/Case_study_submissions/submission_"+"AM_FE",index=False)

# Model Scores

In [ ]:
from tabulate import tabulate
table = [['model', 'data', 'R2_Train_Score' , 'R2_private_score' , 'R2_public_score'], 
         ['Lasso Regressor', "Top_100_Binary_&_7cat_OHT"  , 0.3681, 0.37208,0.37404],
         ['Lasso Regressor','Original_features_&_PCA,SVD_component10' ,0.4383  , 0.4108 ,0.41634],
         ["Lasso Regressor" ,'Original_binary_&_LE_cat_feat' ,0.37284 , 0.37610 , 0.38550],

         ['Decision Tree', "Top_100_Binary_&_7cat_OHT"  ,0.5806,0.5438 ,0.54620],
         ['Decision Tree','Original_features_&_PCA,SVD_component10' ,0.58605 ,0.53522 ,0.56041],
         ["Decision Tree" ,'Original_binary_&_LE_cat_feat' ,0.6172 ,0.51651 ,0.53424] , 

         ['Random Forest' , 'Top_100_Binary_&_7cat_OHT' ,0.5870 ,0.54616 ,0.5505] ,
         ['Random Forest' ,'Original_features_&_PCA,SVD_component10' ,0.5898 ,0.54119 , 0.55309] , 
         ['Random Forest' ,'Original_binary_&_LE_cat_feat',0.5838 , 0.54363 , 0.55400] ,

         ['XGBoost regressor' , 'Top_100_Binary_&_7cat_OHT' ,0.6881 , 0.52547 , 0.55113],
         ['XGBoost regressor' , 'Original_features_&_PCA,SVD_component10' ,0.6492 , 0.31139 , 0.3412],
         ['XGBoost regressor' ,'Original_binary_&_LE_cat_feat',0.9982 , 0.42864 , 0.32162 ] ,

         ['Custom Ensemble_diff', "Top_100_Binary_&_7cat_OHT"  , 0.5885 , 0.53501,0.54139],
         ['Custom Ensemble_diff','Original_features_&_PCA,SVD_component10' ,0.5649  ,0.53225 ,0.55880],
         ["Custom Ensemble_diff" ,'Original_binary_&_LE_cat_feat' ,0.5944 ,0.54340 ,0.54611] ,

         ['Custom Ensemble_same','Original_features_&_PCA,SVD_component10' ,0.5837  ,0.52991 ,0.53663],
         ["Custom Ensemble_same" ,'Original_binary_&_LE_cat_feat' ,0.5854 ,0.53662 ,0.54199] ,

         ['AVG_Model_XGB' , 'Original_binary_&_OHT_cat_feat' ,0.61 , 0.55201 ,0.55446] ,
         ['AVG_Model_XGB' , 'Original_features_&_PCA,SVD_component10' ,0.6373 , 0.54226 ,0.55908]]

print(tabulate(table, headers='firstrow', tablefmt='grid'))

+----------------------+-----------------------------------------+------------------+--------------------+-------------------+
| model                | data                                    |   R2_Train_Score |   R2_private_score |   R2_public_score |
+======================+=========================================+==================+====================+===================+
| Lasso Regressor      | Top_100_Binary_&_7cat_OHT               |          0.3681  |            0.37208 |           0.37404 |
+----------------------+-----------------------------------------+------------------+--------------------+-------------------+
| Lasso Regressor      | Original_features_&_PCA,SVD_component10 |          0.4383  |            0.4108  |           0.41634 |
+----------------------+-----------------------------------------+------------------+--------------------+-------------------+
| Lasso Regressor      | Original_binary_&_LE_cat_feat           |          0.37284 |            0.3761  |     